In [1]:
## Instalaciones

%pip install torch
%pip install open3d
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Dependencias

from typing import List
import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointNetClassifier, PointNetLoss, PointNetKAN
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict
from utils.transformation import (Normalization,
                                  Rotation, Translation, Reflection, Scale,
                                  DropRandom, DropSphere, Jittering, Noise)
from trainer import PointNetTrainer


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using cuda.


In [ ]:
# parámetros del dataset
classes = [label for label in ModelNetClass]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparámetros
num_global_feats = 1024     # número de features globales calculadas
learning_rate = 0.001
reg_weight = 0.001
gamma = 2                   # Recomendado por el paper de focal loss

# train dataset
transformations = [Normalization(),
                   Rotation(), Reflection(), Scale(max_ratio=2.0),
                   DropRandom(), Jittering(), Noise(),
                   Normalization()]

train_data = ModelNet(classes, DatasetType.TRAIN, repetitions=3, transformations=transformations)
validation_data = ModelNet(classes, DatasetType.VALIDATION, repetitions=3, transformations=transformations)
    
# TODO: Más adelante usar alpha para clases imbalanceadas

In [ ]:
# Función de entrenamiento
def train(
        epochs: int,
        name: str,
        num_global_feats: int,
        learning_rate: int,
        use_scheduler: bool,
        alpha: List[int],
        gamma: int,
        reg_weight: int,
        use_kan: bool,
        ignore_Tnet: bool,
):
    if not use_kan:
        classifier = PointNetClassifier(dim, num_points, num_global_feats, num_classes, ignore_Tnet=ignore_Tnet).to(DEVICE)
    else:
        classifier = PointNetKAN(dim, num_points, num_classes, scaling = 3.0).to(DEVICE)
    optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
    if DEVICE == "cuda" and use_scheduler:
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
    else:
        scheduler = None
    
    trainer = PointNetTrainer(
        name=name,
        model=classifier,
        optimizer=optimizer,
        scheduler=scheduler,
        criterion=PointNetLoss(alpha=alpha, gamma=gamma, reg_weight=reg_weight, size_average=True).to(DEVICE),
        device=DEVICE,
        train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True),
        val_loader=DataLoader(validation_data, batch_size=batch_size, shuffle=False),
        checkpoint_dir=os.path.join(os.getcwd(), "checkpoint"),
        checkpoint_freq=25
    )

    loss_dict, best_epoch, best_loss, best_acc = trainer.fit(epochs=epochs)
    #save_loss_dict(loss_dict, os.path.join(os.getcwd(), "csv", f"{name}_loss_dict.csv"))
    print(f"{name} | Best model @ epoch {best_epoch}: loss = {best_loss:.4f}, acc = {best_acc:.4f}")

# Instancias de entrenamiento
EPOCHS=100

In [5]:
train(epochs=EPOCHS, name="base", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

╒═════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch   │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1 │       2.3948 │      0.3209 │    12.9977 │    0.1238 │
├─────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2 │       1.8154 │      0.4622 │     3.0394 │    0.1975 │
╘═════════╧══════════════╧═════════════╧════════════╧═══════════╛
base | Best model @ epoch 2: loss = 3.0394, acc = 0.1975


In [6]:
alpha = [3991/106, 3991/515, 3991/889, 3991/200, 3991/200, 3991/465, 3991/200, 3991/680, 3991/392, 3991/344]
train(epochs=EPOCHS, name="alpha", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=alpha, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

╒═════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch   │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1 │      22.0721 │      0.2426 │    669.356 │    0.1138 │
├─────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2 │      19.5657 │      0.3301 │    529.306 │    0.1162 │
╘═════════╧══════════════╧═════════════╧════════════╧═══════════╛
alpha | Best model @ epoch 2: loss = 529.3061, acc = 0.1163


In [7]:
train(epochs=EPOCHS, name="scheduler", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=True, alpha=None, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

╒═════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch   │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1 │       2.2635 │      0.3143 │    13.5566 │    0.235  │
├─────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2 │       1.9786 │      0.4187 │     6.8976 │    0.2212 │
╘═════════╧══════════════╧═════════════╧════════════╧═══════════╛
scheduler | Best model @ epoch 2: loss = 6.8976, acc = 0.2213


In [8]:
train(epochs=EPOCHS, name="gamma", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=gamma, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

╒═════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch   │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1 │       1.8439 │      0.3494 │    18.3526 │    0.0946 │
├─────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2 │       1.4693 │      0.4648 │     9.2847 │    0.1875 │
╘═════════╧══════════════╧═════════════╧════════════╧═══════════╛
gamma | Best model @ epoch 2: loss = 9.2847, acc = 0.1875


In [9]:
train(epochs=EPOCHS, name="noreg", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=0, use_kan=False, ignore_Tnet=False)

╒═════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch   │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1 │       1.9595 │      0.3346 │    68.6742 │      0.22 │
├─────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2 │       1.6029 │      0.4654 │    69.0752 │      0.28 │
╘═════════╧══════════════╧═════════════╧════════════╧═══════════╛
noreg | Best model @ epoch 1: loss = 68.6742, acc = 0.2200


In [10]:
train(epochs=EPOCHS, name="kan", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=0, use_kan=True, ignore_Tnet=False)

╒═════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch   │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1 │       3.886  │      0.4616 │    12.6383 │    0.2425 │
├─────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2 │       3.0235 │      0.5616 │    12.9663 │    0.1762 │
╘═════════╧══════════════╧═════════════╧════════════╧═══════════╛
kan | Best model @ epoch 1: loss = 12.6383, acc = 0.2425


In [11]:
train(epochs=EPOCHS, name="noTnet", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=0, use_kan=False, ignore_Tnet=True)

╒═════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch   │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1 │       1.1895 │      0.6337 │    244.642 │    0.1275 │
├─────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2 │       0.7177 │      0.7804 │    681.951 │    0.0712 │
╘═════════╧══════════════╧═════════════╧════════════╧═══════════╛
noTnet | Best model @ epoch 1: loss = 244.6415, acc = 0.1275


In [ ]:
train(epochs=EPOCHS, name="global", num_global_feats=512, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

In [12]:
raise Exception("dark souls 2: scholar of the first sin")

Exception: dark souls 2: scholar of the first sin

In [ ]:
"""
# Clasificador con convolución unidimensional
base_classifier = PointNetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)
base_optimizer = optim.Adam(base_classifier.parameters(), lr=learning_rate)
if DEVICE == "cuda":
    base_scheduler = torch.optim.lr_scheduler.CyclicLR(base_optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
else:
    base_scheduler = None

base_trainer = PointNetTrainer(
    name="pointnet_base",
    model=base_classifier,
    optimizer=base_optimizer,
    scheduler=base_scheduler,
    criterion=PointNetLoss(alpha=None, gamma=gamma, reg_weight=reg_weight, size_average=True).to(DEVICE),
    device=DEVICE,
    train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True),
    val_loader=DataLoader(validation_data, batch_size=batch_size, shuffle=False),
    checkpoint_dir=os.path.join(os.getcwd(), "checkpoint"),
    checkpoint_freq=25
)

# Clasificador con KAN
kan_classifier = PointNetKAN(dim, num_classes, scaling = 3.0).to(DEVICE)
kan_optimizer = optim.Adam(kan_classifier.parameters(), lr=learning_rate)
if DEVICE == "cuda":
    kan_scheduler = torch.optim.lr_scheduler.CyclicLR(kan_optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
else:
    kan_scheduler = None
kan_trainer = PointNetTrainer(
    name="pointnet_kan",
    model=kan_classifier,
    optimizer=kan_optimizer,
    scheduler=kan_scheduler,
    criterion=PointNetLoss(alpha=None, gamma=gamma, reg_weight=0, size_average=True).to(DEVICE),
    device=DEVICE,
    train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True),
    val_loader=DataLoader(validation_data, batch_size=batch_size, shuffle=False),
    checkpoint_dir=os.path.join(os.getcwd(), "checkpoint"),
    checkpoint_freq=25
)
"""

In [ ]:
"""
# loss_dict, best_epoch, best_loss, best_acc = base_trainer.fit(epochs=500)
loss_dict, best_epoch, best_loss, best_acc = kan_trainer.fit(epochs=500)
save_loss_dict(loss_dict)
print(f"Best model @ epoch {best_epoch}: loss = {best_loss:.4f}, acc = {best_acc:.4f}")
"""

# Dataset Testeo

In [ ]:
# test dataset
base_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Normalization()])
base_test_loader = DataLoader(base_test_data, batch_size=batch_size, shuffle=False)

affine_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Normalization(),
                                           Rotation(), Reflection(), Scale(max_ratio=2.0),
                                           Normalization()])
affine_test_loader = DataLoader(affine_test_data, batch_size=batch_size, shuffle=False)

complex_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Normalization(),
                                           Rotation(), Reflection(), Scale(max_ratio=2.0),
                                           DropRandom(), Jittering(), Noise(),
                                           Normalization()])
comples_test_loader = DataLoader(complex_test_data, batch_size=batch_size, shuffle=False)

In [ ]:
with torch.no_grad():
    classifier = classifier.eval()
    correct = 0
    
    for pcds, labels in test_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)
        
        # Hacer predicciones
        out, _, _ = classifier(pcds)
    
        # Calculamos las elecciones
        pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
        
        # Elecciones correctas, acumuladas
        correct += pred_choice.eq(labels.data).cpu().sum().item()

    test_acc = correct / float(len(test_data))
    print("Test Acc:\t", test_acc)

In [ ]:
def test_it(path):
    classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)
    classifier.load_state_dict(torch.load(path))

    with torch.no_grad():
        classifier = classifier.eval()
        correct = 0
        
        for pcds, labels in test_loader:
            pcds = pcds.to(DEVICE)
            labels = labels.squeeze().to(DEVICE)
            
            # Hacer predicciones
            out, _, _ = classifier(pcds)
        
            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas, acumuladas
            correct += pred_choice.eq(labels.data).cpu().sum().item()

        test_acc = correct / float(len(test_data))
        print(f"{path}\t\t", test_acc)

# Tests
for i in range(25, 500 + 1, 25):
    path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{str(i).zfill(4)}.pth")
    test_it(path)
path = os.path.join(CHECKPOINT_DIR, "best_model.pth")
test_it(path)